In [ ]:
def preprocess(df_test):
    is_empty = (df_test['hc'].isnull() & df_test['hcnox'].notnull() & df_test['nox'].notnull())

    # Remplir les valeurs manquantes dans "hcnox" avec la somme de "hc" et "nox"
    df_test.loc[is_empty,'hc'] = df_test.loc[is_empty,'hcnox'] - df_test.loc[is_empty,'nox']
    df_test = df_test.drop(columns=["id","hcnox","model"])

    df_test[['gearbox', 'ratio']] = df_test["grbx_type_ratios"].str.split(' ', n=1, expand=True)
    df_test['ratio'] = df_test['ratio'].replace({'.': '0'})                                          #Remplacer les points par zéro
    df_test['ratio'] = pd.to_numeric(df_test['ratio'], errors='coerce').fillna(0)
    df_test = df_test.drop(columns="grbx_type_ratios")

    is_not_abundant = ~df_test["brand"].isin(["MERCEDES", "VOLKSWAGEN"])
    df_test.loc[is_not_abundant, "brand"] = "AUTRE"

    return df_test

In [ ]:
pipeline.fit(data, target)

df_test_processed = preprocess(df_test)
submissions = pipeline.predict(df_test_processed)

submit = pd.DataFrame({ 'id': df_test["id"], 'co2': submissions })
submit.to_csv("submission.csv", index=False)